In [82]:
import pandas as pd

def create_title_release_year_column_for_wiki(x):
    return str(x["Release Year"])+"_"+str(x["Title"])
def create_title_release_year_column_for_movies(x):
    return str(x["release_date"])[0:4]+"_"+str(x["title"])

In [83]:
movies_df = pd.read_csv("Data/movies_dataset/movies_metadata.csv")
wiki_movies_df = pd.read_csv("Data/wikipedia_movies_dataset/wiki_movie_plots_deduped.csv")
keywords_df = pd.read_csv("Data/movies_dataset/keywords.csv")

credits_df = pd.read_csv("Data/movies_dataset/credits.csv")

/tmp/ipykernel_301418/1189048178.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv("Data/movies_dataset/movies_metadata.csv")


In [84]:
credits_df.head()
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [85]:
movies_df['ReleaseAndTitle'] = movies_df.apply(create_title_release_year_column_for_movies, axis=1)
movies_df.shape

(45466, 25)

In [86]:
wiki_movies_df['ReleaseAndTitle'] = wiki_movies_df.apply(create_title_release_year_column_for_wiki, axis=1)
wiki_movies_df.shape

(34886, 9)

In [87]:
df_merged = pd.merge(movies_df, wiki_movies_df, how='inner', left_on="ReleaseAndTitle",right_on="ReleaseAndTitle")
print(df_merged.shape)
#to compare it with the other dataframes
df_merged["id"] = pd.to_numeric(df_merged["id"])



(14495, 33)


In [88]:

df_merged["id"] = pd.to_numeric(df_merged["id"])

#add also keywords
df_merged = pd.merge(df_merged, keywords_df, how='inner', left_on="id",right_on="id")
print(df_merged.shape)

(14676, 34)


In [89]:
#add also credits
df_merged = pd.merge(df_merged, credits_df, how='inner', left_on="id",right_on="id")
print(df_merged.shape)

(14706, 36)


In [90]:
df_merged.head()
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14706 entries, 0 to 14705
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  14706 non-null  object 
 1   belongs_to_collection  1873 non-null   object 
 2   budget                 14706 non-null  object 
 3   genres                 14706 non-null  object 
 4   homepage               2246 non-null   object 
 5   id                     14706 non-null  int64  
 6   imdb_id                14705 non-null  object 
 7   original_language      14706 non-null  object 
 8   original_title         14706 non-null  object 
 9   overview               14692 non-null  object 
 10  popularity             14706 non-null  object 
 11  poster_path            14698 non-null  object 
 12  production_companies   14706 non-null  object 
 13  production_countries   14706 non-null  object 
 14  release_date           14706 non-null  object 
 15  re

In [91]:
df_merged.to_csv("Data/merged.csv")

In [92]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14706 entries, 0 to 14705
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  14706 non-null  object 
 1   belongs_to_collection  1873 non-null   object 
 2   budget                 14706 non-null  object 
 3   genres                 14706 non-null  object 
 4   homepage               2246 non-null   object 
 5   id                     14706 non-null  int64  
 6   imdb_id                14705 non-null  object 
 7   original_language      14706 non-null  object 
 8   original_title         14706 non-null  object 
 9   overview               14692 non-null  object 
 10  popularity             14706 non-null  object 
 11  poster_path            14698 non-null  object 
 12  production_companies   14706 non-null  object 
 13  production_countries   14706 non-null  object 
 14  release_date           14706 non-null  object 
 15  re

In [93]:
rating_df = pd.read_csv("Data/movies_dataset/ratings.csv")
rating_df.shape

(26024289, 4)

In [94]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [95]:
#keep only the ones that are associated with the movies we have in the merged dataset
rating_df = rating_df[rating_df["movieId"].isin(unique_movie_ids)]
rating_df.shape

(6935908, 4)

In [96]:
rating_df.head()
rating_df.to_csv("Data/updated_ratings.csv")

In [169]:
import json
import ast
import re


# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'Director', 'genres']

for feature in features:
    df_merged[feature] = df_merged[feature].apply(clean_data)

def create_soup(x):
    keywords = [k["name"] for k in ast.literal_eval(x['keywords'])]
    cast = [k.replace("'","") for k in re.findall(r"'name':([\w.']+)",x['cast'])]
    genres = [k["name"] for k in ast.literal_eval(x['genres'])]
    return ' '.join(keywords) + ' ' + ' '.join(cast) + ' ' + x['Director'] + ' ' + ' '.join(genres)


df_merged['KeywCastDirGenre'] = df_merged.apply(create_soup, axis=1)



In [178]:
df_merged["KeywCastDirGenre"][25]
df_merged.to_csv("Data/merged.csv")
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6935908 entries, 2 to 26024271
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 264.6 MB


In [193]:
dict35 = dict()
dict4 = dict()

internal_dictionary_mapping = {"title":0,"other":1}
rating_df["userId"].describe()
rating_df35 = rating_df[rating_df["rating"] >= 3.5]
df35 = rating_df35.groupby("userId").agg(list)

# for userid in rating_df["userId"].unique():
#     pass
#     dict35[userid] = ([],[]) #[[Titles of movies rated more than 3.5],['cast', 'keywords', 'director', 'genres']]
#     dict4[userid] = ([],[]) #[[Titles of movies rated more than 4],['cast', 'keywords', 'director', 'genres']]



In [192]:
df35.head()

,userId,movieId,rating,timestamp
2,1,858,5.0,1425941523
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148
6,1,2762,4.5,1425941300
8,1,2959,4.0,1425941601
...,...,...,...,...
26024033,270896,25,4.0,1257034064
26024041,270896,111,5.0,1257034267
26024042,270896,141,3.5,1257033747
26024046,270896,223,4.0,1257033898
